In [17]:
from pgmpy.models import BayesianModel

In [18]:
# Starting with defining the network structure
uav_model = BayesianModel([('guidance action','navigation'),
                           ('guidance action','training'),
                           ('guidance action','telemetry'),
                           ('awareness','power plant'),
                           ('awareness','workload'),
                           ('awareness','telemetry'),
                           ('awareness','obstacle sensing'),
                           ('awareness','weather'),
                           ('knowledge','telemetry'),
                           ('knowledge','training'),
                           ('knowledge','ADS-B'),
                           ('mishap','guidance action'),
                           ('mishap','navigation'),
                           ('mishap','power plant'),
                           ('mishap','awareness'),
                           ('mishap','obstacle sensing'),
                           ('mishap','knowledge'),
                           ('mishap','ADS-B'),
                           ('mishap','weather')])

In [19]:
from pgmpy.factors.discrete import TabularCPD


In [16]:
#  Now defining the parameters
cpd_tel = TabularCPD(variable='Telemetry failure', variable_card=2,
                      values=[[0.999995657], [4.343e-6]])
cpd_pow = TabularCPD(variable='Power plant failure', variable_card=2,
                      values=[[0.99999353], [6.47e-6]])
cpd_los = TabularCPD(variable='Operator loss of situation awareness', variable_card=2,
                        values=[[0.99996809, 4.813e-6, 0.976, 0],
                                [3.191e-5, 0.999995187, 2.4e-2, 1]],
                        evidence=['Telemetry failure', 'Power plant failure'],
                        evidence_card=[2, 2])
cpd_mis = TabularCPD(variable='UAV mishaps', variable_card=2,
                      values=[[0,3.191e-5,1,1],
                              [1,0.99996809,0,0]],
                      evidence=['Operator loss of situation awareness','Power plant failure'], 
                      evidence_card=[2,2])

In [6]:
# Associating the parameters with the model structure.
uav_model.add_cpds(cpd_tel, cpd_pow, cpd_los, cpd_mis)

# Checking if the cpds are valid for the model.
uav_model.check_model()

True

In [7]:
uav_model.get_independencies()

(Telemetry failure _|_ Power plant failure)
(Telemetry failure _|_ UAV mishaps | Operator loss of situation awareness, Power plant failure)
(Power plant failure _|_ Telemetry failure)
(UAV mishaps _|_ Telemetry failure | Operator loss of situation awareness, Power plant failure)

In [9]:
uav_model.get_cpds()

[<TabularCPD representing P(Telemetry failure:2) at 0x125092af0>,
 <TabularCPD representing P(Power plant failure:2) at 0x125092640>,
 <TabularCPD representing P(Operator loss of situation awareness:2 | Telemetry failure:2, Power plant failure:2) at 0x125092670>,
 <TabularCPD representing P(UAV mishaps:2 | Operator loss of situation awareness:2, Power plant failure:2) at 0x125092610>]

In [13]:
from pgmpy.inference import VariableElimination
uav_infer = VariableElimination(uav_model)

In [14]:
mishap_infer = uav_infer.query(variables = ['UAV mishaps'])
print(mishap_infer['UAV mishaps'])

Eliminating: Power plant failure: 100%|██████████| 3/3 [00:00<00:00, 180.02it/s]


TypeError: 'DiscreteFactor' object is not subscriptable